In [231]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [232]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [258]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
import pprint

warnings.filterwarnings("ignore")


In [238]:
def drop_unwanted_columns(df):
    df = df.drop('teamID', axis=1)
    df = df.drop('inhibKills', axis=1)
    df = df.drop('towerKills', axis=1)
    df = df.drop('assists', axis=1)
    df = df.drop('game_time', axis=1)
    return df

In [239]:
with open('esports-data/lck_2023_average_stats.json', 'r') as json_file:
    data = json.load(json_file)

df = pd.DataFrame(data)


In [240]:
df = drop_unwanted_columns(df)

In [241]:
df.shape

(470, 10)

In [242]:
df

,baronKills,championsKills,deaths,dragonKills,first_blood,first_turret,totalDamage,totalGold,totalLevel,won
0,0,0.745937,0.043879,3,True,True,22695.472874,1948.695597,2.676599,True
1,0,0.043879,0.745937,0,False,False,16546.005534,1438.693820,2.457205,False
2,2,0.416064,0.388326,2,False,False,30703.887080,1916.335087,2.246745,True
3,0,0.388326,0.416064,3,True,True,23298.979904,1764.471744,2.219008,False
4,0,0.223992,0.671976,0,False,False,20090.998288,1647.796386,2.426579,False
...,...,...,...,...,...,...,...,...,...,...
465,1,0.097045,0.291136,2,False,False,27738.625310,1664.738566,2.013689,False
466,0,0.266911,0.762602,1,True,False,20916.845439,1693.699831,2.440325,False
467,1,0.762602,0.266911,2,False,True,25539.891611,2037.251813,2.745365,True
468,0,0.200154,0.533745,0,True,False,24639.841918,1735.471575,2.335134,False


In [243]:
from sklearn.preprocessing import OrdinalEncoder

# Extract feature and target arrays
X, y = df.drop('won', axis=1), df[['won']]
# y = OrdinalEncoder().fit_transform(y)

In [244]:
y

,won
0,True
1,False
2,True
3,False
4,False
...,...
465,False
466,False
467,True
468,False


In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [246]:
# Create regression matrices
dtrain_clf = xgb.DMatrix(X_train, y_train)
dtest_clf = xgb.DMatrix(X_test, y_test)

In [247]:
evals = [(dtrain_clf, "train"), (dtest_clf, "validation")]


xgb_classifier = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3, evals=evals)
xgb_classifier.fit(X_train, y_train)


model = xgb_classifier.get_booster()


In [248]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_clf)

In [249]:
rmse = mean_squared_error(y_test, preds, squared=False)

In [250]:
print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 0.103


In [251]:
new_data = [
{
    "inhibKills": 0,
    "towerKills": 1,
    "teamID": 200,
    "baronKills": 0,
    "dragonKills": 0,
    "assists": 0.08775733925597866,
    "totalGold": 1438.693819762514,
    "championsKills": 0.04387866962798933,
    "deaths": 0.7459373836758185,
    "totalDamage": 16546.005533877258,
    "totalLevel": 2.457205499167402,
    "won": False,
    "first_blood": False,
    "first_turret": False,
    "game_time": 1367407
  }
]

test = pd.DataFrame(new_data)
test = drop_unwanted_columns(test)
test = test.drop('won', axis = 1)

help = xgb.DMatrix(test)

predictions = model.predict(help)


ValueError: feature_names mismatch: ['baronKills', 'championsKills', 'deaths', 'dragonKills', 'first_blood', 'first_turret', 'totalDamage', 'totalGold', 'totalLevel'] ['baronKills', 'dragonKills', 'totalGold', 'championsKills', 'deaths', 'totalDamage', 'totalLevel', 'first_blood', 'first_turret']

In [252]:
predictions

array([0.00121212], dtype=float32)

In [256]:
with open('esports-data/lck_2023_team_average_stats.json', 'r') as json_file:
    testing_data = json.load(json_file)

with open('esports-data/teams_but_better.json', 'r') as json_file:
    team_names = json.load(json_file)


result_dict = dict()

for team_name, team_stats in testing_data.items():
    count = 0
    prediction = 0
    for entry in team_stats:
        test = pd.DataFrame([entry])
        test = drop_unwanted_columns(test)
        test = test.drop('won', axis = 1)
        test_matrix = xgb.DMatrix(test)
        prediction += model.predict(test_matrix)
        count += 1
    prediction /= count
    result_dict[team_names[team_name]] = prediction





In [260]:
pprint.pprint(sorted(result_dict.items(), key=lambda x:x[1], reverse=True))


[('Gen.G', array([0.8107719], dtype=float32)),
 ('kt Rolster', array([0.77995384], dtype=float32)),
 ('Hanwha Life Esports', array([0.59992224], dtype=float32)),
 ('Dplus Kia', array([0.53384155], dtype=float32)),
 ('T1', array([0.50062335], dtype=float32)),
 ('Liiv SANDBOX', array([0.34214368], dtype=float32)),
 ('DRX', array([0.33464232], dtype=float32)),
 ('OKSavingsBank BRION', array([0.331718], dtype=float32)),
 ('NongShim REDFORCE', array([0.30504218], dtype=float32)),
 ('Kwangdong Freecs', array([0.26372486], dtype=float32))]
